<a href="https://colab.research.google.com/github/danielmohansahu/M3DETR/blob/kitti_downsample/evaluation_dwnsmpl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# M3DETR Evaluation Notebook

This notebook demonstrates evaluation of the [M3DETR](https://arxiv.org/pdf/2104.11896.pdf) 3D Object Detection model architecture and training pipeline. Specifically it supports training and inference of [Group 13's fork](https://github.com/danielmohansahu/M3DETR). All training and evaluation uses the [KITTI 3D Detection Evaluation Dataset](https://www.cvlibs.net/datasets/kitti/eval_object.php?obj_benchmark=3d).

Note - this notebook requires memory resources beyond those available in the current free tiers of Google services. In order to run this script you will need the following:
 1. Google Colab Pro account
 2. A Google Drive with at least 50GB of free space (to cache the processed KITTI Dataset)

Assuming those are satisfied, please change the runtime to a GPU-enabled (T4 was used during this testing) and continue.

In [1]:
# sanity check hardware
!nvidia-smi

# connect to google drive
from google.colab import drive
drive.mount('/content/drive')

# sanity check required packages are installed
import torch
import torchvision
import pathlib
import shutil

# install M3DETR dependencies
!pip install spconv-cu120 cython

Fri Nov 10 04:48:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%%time

# clone repository
!git clone https://github.com/danielmohansahu/M3DETR.git
%cd M3DETR

# build repository
!python3 setup.py develop

Cloning into 'M3DETR'...
remote: Enumerating objects: 614, done.
remote: Counting objects: 100% (317/317), done.
remote: Compressing objects: 100% (182/182), done.
remote: Total 614 (delta 155), reused 249 (delta 121), pack-reused 297
Receiving objects: 100% (614/614), 58.77 MiB | 22.46 MiB/s, done.
Resolving deltas: 100% (224/224), done.
/content/M3DETR
running develop
/usr/local/lib/python3.10/dist-packages/setuptools/command/develop.py:40: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://github.com/pypa/setuptools/issues/917 for details.
        ********************************************************************************

!!
  easy_install.initialize_options(self)
/usr/local/lib/python3.1

## KITTI Dataset Prep

The following module uses `fiftyone` to download and pre-process the KITTI 3D Object Detection dataset.

Due to the large size (~50GB) and significant amount of time required we attempt to sync this processed data to and from our Google Drive. Otherwise we would spend a non-trivial percentage of our allotted Google Colab runtime repeating this same process.

Note: this is also the main reason we need to use Google Colab Pro (and an upgraded Google Drive) - the space requirements for even a relatively small dataset like KITTI are beyond those provided by the free Google services.

This requires the user to create a `cache` directory in the root of their Google Drive. They must also have roughly 50GB of free space available on their drive. Please be prepared to spend several hours monitoring this process to ensure it succeeds and properly uploads the cached dataset.

In [3]:
%%time

# local location of drive cache (mounted)
cache_file = pathlib.Path("/content/drive/MyDrive/cache/m3detr/kitti.zip")

# local root directory of data
data_root_dir = pathlib.Path("/content/M3DETR/data")

# local location of dataset
data_dir = data_root_dir / "kitti"

if not cache_file.exists():
  # download and fully process KITTI from scratch. this will be slow!

  # install dataset management helper
  !python3 -m pip install fiftyone
  !python3 -m pip install fiftyone-db-ubuntu2204

  # import dataset management helper
  import fiftyone

  print("Downloading and processing KITTI dataset...")
  # note - this is kind of a hack to avoid a slow unnecessary operation
  try:
    dataset = fiftyone.zoo.load_zoo_dataset('kitti-multiview', dataset_dir=data_dir.as_posix())
  except ImportError:
    print("Skipping converting PointClouds to PCD; this is unnecessary.")
  else:
    raise RuntimeError("We shouldn't succeed here - you're going to run out of space!")

  # rename folders to follow expected convention
  print("Renaming and removing unneeded files / folders...")
  !cd {data_dir.as_posix()} && mv test testing && mv train training
  !cd {data_dir.as_posix()} && mv testing/left testing/image_2
  !cd {data_dir.as_posix()} && mv training/left training/image_2 && mv training/labels training/label_2

  # remove those we don't care about
  !cd {data_dir.as_posix()} && rm -rf tmp-download testing/right training/right

  # post-process via OpenPCD scripts
  print("Post-processing dataset for OpenPCD...")
  !python3 -m pcdet.datasets.kitti.kitti_dataset create_kitti_infos \
              tools/cfgs/dataset_configs/kitti_dataset.yaml

  # zip up the results for faster copy speeds
  print("Archiving dataset...")
  !cd {data_root_dir.as_posix()} && zip kitti.zip kitti -r

  # finally, copy over results to drive
  print("Backing up to drive...")
  !cd {data_root_dir.as_posix()} && mv kitti.zip {cache_file.as_posix()}

elif not (data_dir / "training").is_dir() or not (data_dir / "testing").is_dir():
  # pull pre-processed KITTI dataset from local drive

  print("Copying over cached KITTI dataset...")
  !cp {cache_file.as_posix()} {data_root_dir.as_posix()}

  print("Unzipping cached KITTI dataset...")
  !cd {data_root_dir.as_posix()} && unzip -o kitti.zip

# sanity check before continuing
for directory in ("calib", "label_2", "image_2", "velodyne"):
  assert (data_dir / "training" / directory).is_dir(), f"Missing required training data : {directory}"
for directory in ("calib", "image_2", "velodyne"):
  assert (data_dir / "testing" / directory).is_dir(), f"Missing required testing data : {directory}"
assert (data_dir / "kitti_dbinfos_train.pkl").is_file(), "Missing required metadata!"
assert (data_dir / "kitti_infos_test.pkl").is_file(), "Missing required metadata!"
assert (data_dir / "kitti_infos_train.pkl").is_file(), "Missing required metadata!"
assert (data_dir / "kitti_infos_trainval.pkl").is_file(), "Missing required metadata!"
assert (data_dir / "kitti_infos_val.pkl").is_file(), "Missing required metadata!"
print("KITTI dataset loaded.")

Streaming output truncated to the last 5000 lines.
  inflating: kitti/training/velodyne/002484.bin  
  inflating: kitti/training/velodyne/002485.bin  
  inflating: kitti/training/velodyne/002486.bin  
  inflating: kitti/training/velodyne/002487.bin  
  inflating: kitti/training/velodyne/002488.bin  
  inflating: kitti/training/velodyne/002489.bin  
  inflating: kitti/training/velodyne/002490.bin  
  inflating: kitti/training/velodyne/002491.bin  
  inflating: kitti/training/velodyne/002492.bin  
  inflating: kitti/training/velodyne/002493.bin  
  inflating: kitti/training/velodyne/002494.bin  
  inflating: kitti/training/velodyne/002495.bin  
  inflating: kitti/training/velodyne/002496.bin  
  inflating: kitti/training/velodyne/002497.bin  
  inflating: kitti/training/velodyne/002498.bin  
  inflating: kitti/training/velodyne/002499.bin  
  inflating: kitti/training/velodyne/002500.bin  
  inflating: kitti/training/velodyne/002501.bin  
  inflating: kitti/training/velodyne/002502.bin  

## Evaluation

The following cell downloads a pre-trained model provided by the M3DETR authors and uses it to run inference against the KITTI test split.

My output:

```bash
Car AP@0.70, 0.70, 0.70:
bbox AP:90.4188, 87.2058, 86.5413
bev  AP:89.4501, 83.9316, 78.8657
3d   AP:86.7184, 75.8649, 74.6789
aos  AP:90.38, 87.06, 86.29
Car AP_R40@0.70, 0.70, 0.70:
bbox AP:95.7361, 88.6217, 88.4904
bev  AP:92.1735, 84.3550, 82.5889
3d   AP:88.0686, 76.3739, 74.2172
aos  AP:95.68, 88.47, 88.23
Car AP@0.70, 0.50, 0.50:
bbox AP:90.4188, 87.2058, 86.5413
bev  AP:90.4156, 87.8814, 87.6633
3d   AP:90.4156, 87.7556, 87.4215
aos  AP:90.38, 87.06, 86.29
Car AP_R40@0.70, 0.50, 0.50:
bbox AP:95.7361, 88.6217, 88.4904
bev  AP:95.7337, 90.7920, 90.7031
3d   AP:95.7115, 90.5816, 88.8729
aos  AP:95.68, 88.47, 88.23
Pedestrian AP@0.50, 0.50, 0.50:
bbox AP:69.0836, 63.5830, 58.7152
bev  AP:65.9262, 57.3184, 53.6799
3d   AP:63.5804, 55.4299, 49.8286
aos  AP:64.99, 59.24, 54.57
Pedestrian AP_R40@0.50, 0.50, 0.50:
bbox AP:70.6837, 62.6881, 58.4845
bev  AP:66.0663, 56.6501, 51.7529
3d   AP:63.5731, 54.2267, 49.1444
aos  AP:66.08, 58.18, 53.92
Pedestrian AP@0.50, 0.25, 0.25:
bbox AP:69.0836, 63.5830, 58.7152
bev  AP:75.7598, 68.1101, 65.2803
3d   AP:75.6912, 67.9892, 65.0941
aos  AP:64.99, 59.24, 54.57
Pedestrian AP_R40@0.50, 0.25, 0.25:
bbox AP:70.6837, 62.6881, 58.4845
bev  AP:77.0143, 69.3589, 65.0146
3d   AP:76.9322, 69.1994, 64.8358
aos  AP:66.08, 58.18, 53.92
Cyclist AP@0.50, 0.50, 0.50:
bbox AP:88.7362, 69.4412, 68.4013
bev  AP:87.6113, 66.6779, 64.6245
3d   AP:85.4439, 64.5812, 62.6526
aos  AP:88.39, 68.93, 67.83
Cyclist AP_R40@0.50, 0.50, 0.50:
bbox AP:92.1071, 72.0177, 69.4013
bev  AP:90.7867, 67.4259, 64.5058
3d   AP:88.2604, 64.9611, 60.8926
aos  AP:91.72, 71.42, 68.75
Cyclist AP@0.50, 0.25, 0.25:
bbox AP:88.7362, 69.4412, 68.4013
bev  AP:87.7892, 66.9322, 65.7805
3d   AP:87.7892, 66.9322, 65.7805
aos  AP:88.39, 68.93, 67.83
Cyclist AP_R40@0.50, 0.25, 0.25:
bbox AP:92.1071, 72.0177, 69.4013
bev  AP:92.8184, 69.0203, 66.4031
3d   AP:92.8184, 69.0203, 66.4031
aos  AP:91.72, 71.42, 68.75
```

In [8]:
%%time

# download pre-trained model
!mkdir -p /content/models
!cd /content/models && gdown 1Jwr9keDHVabebtf-ApSs7BH8-RDQIUmj

# change to tools directory going forward
%cd /content/M3DETR/tools

# evaluate
!python3 test.py --cfg_file ./cfgs/kitti_models/M3DETR.yaml --workers 1 \
  --ckpt /content/models/m3detr_kitti2.pth --eval_tag evaluation --batch_size 16

Downloading...
From: https://drive.google.com/uc?id=1Jwr9keDHVabebtf-ApSs7BH8-RDQIUmj
To: /content/models/m3detr_kitti2.pth
100% 174M/174M [00:00<00:00, 272MB/s]
/content/M3DETR/tools
2023-11-10 05:28:27,374   INFO  **********************Start logging**********************
2023-11-10 05:28:27,374   INFO  CUDA_VISIBLE_DEVICES=ALL
2023-11-10 05:28:27,374   INFO  cfg_file         ./cfgs/kitti_models/M3DETR.yaml
2023-11-10 05:28:27,374   INFO  batch_size       16
2023-11-10 05:28:27,374   INFO  workers          1
2023-11-10 05:28:27,374   INFO  extra_tag        default
2023-11-10 05:28:27,374   INFO  ckpt             /content/models/m3detr_kitti2.pth
2023-11-10 05:28:27,374   INFO  launcher         none
2023-11-10 05:28:27,374   INFO  tcp_port         18888
2023-11-10 05:28:27,374   INFO  local_rank       0
2023-11-10 05:28:27,374   INFO  set_cfgs         None
2023-11-10 05:28:27,374   INFO  max_waiting_mins 30
2023-11-10 05:28:27,374   INFO  start_epoch      0
2023-11-10 05:28:27,375   IN

## Training

The following cell kicks off training against the full KITTI dataset.

In [9]:
%%time

# train
!python3 train.py --cfg_file ./cfgs/kitti_models/M3DETR.yaml --workers 1 --batch_size 4

Streaming output truncated to the last 5000 lines.
epochs:  75% 30/40 [4:07:37<1:21:23, 488.30s/it, loss=1.21, lr=0.00345, d_time=0.00(0.01), f_time=1.19(1.20), b_time=1.65(1.66)]
train:  42% 104/250 [03:22<04:44,  1.95s/it, total_it=7603]
epochs:  75% 30/40 [4:07:39<1:21:23, 488.30s/it, loss=1.16, lr=0.00345, d_time=0.00(0.01), f_time=1.16(1.20), b_time=1.62(1.66)]
train:  42% 105/250 [03:24<04:41,  1.94s/it, total_it=7604]
epochs:  75% 30/40 [4:07:41<1:21:23, 488.30s/it, loss=1.14, lr=0.00345, d_time=0.00(0.01), f_time=1.17(1.20), b_time=1.63(1.66)]
train:  42% 106/250 [03:26<04:38,  1.93s/it, total_it=7605]
epochs:  75% 30/40 [4:07:42<1:21:23, 488.30s/it, loss=1.15, lr=0.00344, d_time=0.00(0.01), f_time=1.17(1.20), b_time=1.63(1.66)]
train:  43% 107/250 [03:28<04:35,  1.93s/it, total_it=7606]
epochs:  75% 30/40 [4:07:44<1:21:23, 488.30s/it, loss=1.08, lr=0.00344, d_time=0.00(0.01), f_time=1.16(1.20), b_time=1.61(1.66)]
train:  43% 108/250 [03:30<04:34,  1.94s/it, total_it=7607]
epoc